# SANS2D: I(Q) for sample and background

In this notebook, we will be reducing a sample and a background measurements to a one-dimensional $I(Q)$.

It assumes the detector data has been recorded in event mode, while the monitor data has been histogrammed.

The data used in this notebook has been published in [Manasi et al. (2021)](#manasi2021),
and we kindly thank the authors for allowing us to use their data.

**Outline:**

- We will begin by loading the data files containing the sample, direct, and background measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Finally, the sample and background measurement will be converted to the $Q$ dimension

In [ ]:
import matplotlib.pyplot as plt
import scipp as sc
from ess import loki, sans
from ess.logging import configure_workflow
import scippneutron as scn

In [ ]:
logger = configure_workflow('sans2d_reduction', filename='sans2d.log')

## Define reduction workflow parameters

We define here whether to include the effects of gravity,
as well as common wavelength and $Q$ bins for all the measurements.

In [ ]:
# Include effects of gravity?
gravity = True

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141,
                              unit='angstrom')

# Q binning
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.5, num=141, unit='1/angstrom')

## Loading data files

We load the following files:

- The direct beam function for the main detector (gives detector efficiency as a function of wavelength)
- The sample measurement
- The direct measurement: this is the run with the empty sample holder/cuvette
- The background measurement: this is the run with only the solvent which the sample is placed in

In [ ]:
direct_beam = sc.io.open_hdf5(loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.hdf5'))
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.hdf5'))
direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.hdf5'))
background = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063159.hdf5'))
dg = {'sample': sample, 'direct': direct, 'background': background}

## Pre-process monitor data

We convert the monitor data from time-of-flight to wavelength, remove background noise,
and rebin to the requested wavelength binning using the `preprocess_monitor_data` helper function from the `i_of_q` submodule:

In [ ]:
monitors = {}
for key, da in dg.items():
    monitors[f'{key}-incident'] = da.attrs["monitor2"].value
    monitors[f'{key}-transmission'] = da.attrs["monitor4"].value

# Define range outside of which monitor data is considered to be background
non_background_range = sc.array(dims=['wavelength'], values=[0.7, 17.1], unit='angstrom')

# Run preprocessing
monitors = sans.i_of_q.preprocess_monitor_data(
    monitors,
    non_background_range=non_background_range,
    wavelength_bins=wavelength_bins)

# Unpack monitors to make steps below easier
sample_monitors = {'incident': monitors['sample-incident'],
                   'transmission': monitors['sample-transmission']}
direct_monitors = {'incident': monitors['direct-incident'],
                   'transmission': monitors['direct-transmission']}
background_monitors = {'incident': monitors['background-incident'],
                       'transmission': monitors['background-transmission']}

## Masking bad detector pixels

**Note:** We use programmatic masks here and not those stored in xml files.

Now that the monitor data is cleaned and binned to the correct wavelength range, we turn to the detector data.
The first step is to mask noisy and saturated pixels.
We mask the edges of the square-shaped detector panel with a simple distance relation.
We also mask the region close to the beam center,
where the sample holder is visible as a dark patch with an arm extending to the north-east.

In [ ]:
mask_edges = (
    (sc.abs(sample.coords['position'].fields.x) > sc.scalar(0.48, unit='m')) |
    (sc.abs(sample.coords['position'].fields.y) > sc.scalar(0.45, unit='m')))

summed = sample.sum('tof')
holder_mask = (summed.data < sc.scalar(100, unit='counts')) & \
              (sample.coords['position'].fields.x > sc.scalar(0, unit='m')) & \
              (sample.coords['position'].fields.x < sc.scalar(0.42, unit='m')) & \
              (sample.coords['position'].fields.y < sc.scalar(0.05, unit='m')) & \
              (sample.coords['position'].fields.y > sc.scalar(-0.15, unit='m'))

for da in dg.values():
    da.masks['edges'] = mask_edges
    da.masks['holder_mask'] = holder_mask

A good sanity check is to view the masks on the instrument view:

In [ ]:
scn.instrument_view(sample.hist(), pixel_size=0.0075)

### Beam center finder

The beam is not guaranteed to travel through the center of the detector panel,
and we thus have to apply a horizontal and vertical offset to our pixel positions so that the beam centre is at `x = y = 0`.
This is necessary for subsequent azimuthal averaging of the data counts into $Q$ bins.

The `beam_center` utility in the `sans` module is designed for this.
It requires us to define a $Q$ range over which convergence will be checked.

In [ ]:
q_range = sc.linspace('Q', 0.02, 0.3, 71, unit='1/angstrom')

center = sans.beam_center(data=dg['sample'],
                          data_monitors=sample_monitors,
                          direct_monitors=direct_monitors,
                          wavelength_bins=wavelength_bins,
                          q_bins=q_range,
                          gravity=gravity)

print(center)

# Now shift pixels positions to get the correct beam center
for da in dg.values():
    da.coords['position'] -= center

## Mask Bragg peaks in wavelength

We will now take out the time regions with Bragg peaks from the beam stop and detector window,
although in reality the peaks appear only close to the beam stop,
and will make little difference to $I(Q)$.

This could be implemented as masking specific time bins for a specific region in space,
but for now we keep it simple.

In [ ]:
wavelength_mask = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[True]),
    coords={'wavelength': sc.array(dims=['wavelength'], values=[2.21, 2.59], unit='angstrom')})

## Use to_I_of_Q workflow

We now reduce the sample and the background measurements to `Q` using the `sans.to_I_of_Q` workflow.

In that process,
the intensity as a function of `Q` is normalized using the direct measurement and direct beam function.

We call the workflow on both the sample and background runs:

In [ ]:
sample_q = sans.to_I_of_Q(data=dg['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_mask=wavelength_mask)
sample_q.hist().plot()

In [ ]:
background_q = sans.to_I_of_Q(data=dg['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_mask=wavelength_mask)
background_q.hist().plot()

We are now in a position to subtract the background from the sample measurement:

In [ ]:
result = sample_q.bins.sum() - background_q.bins.sum()
result

In [ ]:
fig1, ax1 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(result, ax=ax1[0])
sc.plot(result, norm='log', ax=ax1[1])

<div class="alert alert-info">

**Note**

Instead of `.bins.sum()`,
one could use `sc.histogram()` above to define different `Q` bins compared to the ones defined at the top of the notebook.
This can be done in event mode, see [here](https://scipp.github.io/user-guide/binned-data/computation.html#Subtraction).

There may be performance advantages to first use a coarse `Q` binning when the computing `I(Q)` numerator,
and use finer binning for the final results.

</div>

## Wavelength bands

It is often useful to process the data in a small number (~10) of separate wavelength bands.

This can be achieved by requesting 10 bands from the `to_I_of_Q` workflow via the `wavelength_bands` argument.

In [ ]:
wavelength_bands = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=11,
                               unit='angstrom')

sample_slices = sans.to_I_of_Q(data=dg['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    wavelength_mask=wavelength_mask)

background_slices = sans.to_I_of_Q(data=dg['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    wavelength_mask=wavelength_mask)

result_slices = sample_slices.bins.sum() - background_slices.bins.sum()
result_slices

In [ ]:
collapsed = sc.collapse(result_slices, keep='Q')

fig2, ax2 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(collapsed, ax=ax2[0])
sc.plot(collapsed, norm='log', ax=ax2[1])

## References

<div id="manasi2021"></div>

Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**,
*Self-assembly of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)